In [1]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, multilabel_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from keras_preprocessing.sequence import pad_sequences
from varname import nameof
import seaborn as sns
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import wget

from numpy import asarray
from numpy import savetxt

import geopandas as geop


import glob
import os

from datetime import date

import pymysql

In [2]:
nest_asyncio.apply()

def column_names():
  return twint.output.panda.Tweets_df.columns

def twint_to_pd(columns):
  return twint.output.panda.Tweets_df[columns]

In [4]:
today = date.today()
yesterday = today - timedelta(days = 1)
today=today.strftime('%Y-%m-%d')
yesterday=yesterday.strftime('%Y-%m-%d')
parameter_geo=pd.read_csv("parameter_geo.csv")
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    print(wilayah)
    latitude=parameter_geo.lat_centroid[i]
    print(latitude)
    longitude=parameter_geo.lon_centroid[i]
    print(longitude)
    radius=parameter_geo.radius[i]
    print(radius)
    # param=f"{latitude},{longitude},{radius}"+"km"
    # c= twint.Config()
    # c.Since = yesterday
    # c.Until = today
    # c.Search = "Ganjar Pranowo"
    # # c.Limit=1000
    # c.Geo=param
    # c.Pandas = True
    # twint.run.Search(c)
    # if len(twint.output.panda.Tweets_df.columns) != 0:
    #     data = twint_to_pd(['date','username','tweet'])
    #     data["location"] = wilayah
    #     judul=f"hasil_scraping/{wilayah}_GANJAR.csv"
    #     data.to_csv(judul, index=False)

ACEH
4.226785906634114
96.91391775334364
28978
SUMATERA UTARA
2.188555596320399
99.06660934580025
36490
SUMATERA BARAT
-0.8627272333160265
100.459520785893
21006
RIAU
0.503160835218375
101.80408448052164
43512
JAMBI
-1.6968634433074707
102.72533553832903
25029
SUMATERA SELATAN
-3.208346768007095
104.17363428010638
45796
BENGKULU
-3.571086883536937
102.36137130509093
9960
LAMPUNG
-4.91197147490611
105.01999908326464
17312
KEPULAUAN BANGKA BELITUNG
-2.438783040840763
106.53113977868388
8212
KEPULAUAN RIAU
1.53228807002703
105.50322166565012
4100
DKI JAKARTA
-6.2053450911912185
106.84174820627928
332
JAWA BARAT
-6.921470047105061
107.60228530751726
176
JAWA TENGAH
-7.260815743216024
110.20578520459716
16400
DI YOGYAKARTA
-7.890650167090885
110.4457151065364
1566
JAWA TIMUR
-7.725155777695328
112.71864494032296
23902
BANTEN
-6.454895762196214
106.11516950157292
483
BALI
-8.364767255071614
115.12898433783776
289
NUSA TENGGARA BARAT
-8.610412907979459
117.51543511695962
928
NUSA TENGGARA TIM

In [3]:
today = date.today()
today=today.strftime('%Y-%m-%d')
parameter_geo=pd.read_csv("parameter_geo.csv")
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2023-04-17"
    c.Until = today
    c.Search = "Ganjar Pranowo"
    # c.Limit=1000
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    if len(twint.output.panda.Tweets_df.columns) != 0:
        data = twint_to_pd(['date','username','tweet'])
        data["location"] = wilayah
        judul=f"hasil_scraping/{wilayah}_GANJAR.csv"
        data.to_csv(judul, index=False)

RefreshTokenException: Could not find the Guest token in HTML

In [8]:
df_ganjar= pd.concat(map(pd.read_csv, glob.glob(os.path.join('hasil_scraping/', "*_GANJAR.csv"))))
df_ganjar= df_ganjar.reset_index(drop=True)
df_ganjar.to_csv("data_ganjar.csv")

In [25]:
ganjar=pd.read_csv('data_ganjar.csv')
ganjar=ganjar.drop(columns=['Unnamed: 0'])
ganjar=ganjar.drop_duplicates(subset=['tweet'])
ganjar.tail()

,date,username,tweet,location
78328,2023-04-07 05:52:07,triliuner007,@driveronline_17 @MinmonPS7 @PDI_Perjuangan @D...,SULAWESI UTARA
78329,2023-04-07 05:11:19,triliuner007,@driveronline_17 @MinmonPS7 @PDI_Perjuangan @D...,SULAWESI UTARA
78330,2023-04-07 04:38:38,markhudens,@ganjarpranowo @jokowi Bekerja untuk partai,SULAWESI UTARA
78354,2023-03-31 17:08:10,Vladimermody,Pak @jokowi Mimpimu terlalu besar .. @ganjarpr...,SULAWESI UTARA
78380,2023-02-22 20:46:52,Vladimermody,@oktarayya @KakekHalal Pak @ganjarpranowo aja...,SULAWESI UTARA


In [24]:
# df_ganjar=ganjar.iloc[1001:2000,0:4]
# df_ganjar.tail()

,date,username,tweet,location
2007,2023-04-07 15:33:38,Pakpong77,"@YRadianto @ganjarpranowo Gak Malu, Habis ""MEM...",BALI
2008,2023-04-07 15:28:51,OniMeniq74,@GustiMberkahi @ganjarpranowo Pak Ganjar &amp;...,BALI
2009,2023-04-07 15:28:15,OniMeniq74,@Valent_DK @ganjarpranowo Top Pak Ganjar,BALI
2010,2023-04-07 15:27:55,OniMeniq74,@Lolipopzx_ @ganjarpranowo @jokowi @gibran_twe...,BALI
2011,2023-04-07 15:27:53,Ngalam_wes,@WildaAr_ @ganjarpranowo Sip Pak Ganjar,BALI


In [27]:
# inisiasi variabel awal
start = 0
end = 1000

# loop untuk membagi data menjadi bagian-bagian dengan jumlah baris 1000
while end <= len(ganjar):
    df_ganjar = ganjar.iloc[start:end, 0:4]
    #menyimpan tweet. (tipe data series pandas)
    data_content = df_ganjar['tweet']
    # casefolding
    data_casefolding = data_content.str.lower()
    #data_casefolding.head()
    #filtering

    #url
    filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
    #cont
    filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
    #punctuatuion
    filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
    #  hapus #tagger
    filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
    #numeric
    filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

    # # filtering RT , @ dan #
    # fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
    # clean = [fungsi_clen_rt for tweet in filtering_numeric]

    data_filtering = pd.Series(filtering_numeric)

    # #tokenize
    tknzr = TweetTokenizer()
    data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

    #slang word
    path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
    dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

    def replaceSlang(word):
        if word in list(dataslang[0]):
            indexslang = list(dataslang[0]).index(word)
            return dataslang[1][indexslang]
        else:
            return word

    data_formal = []
    for data in data_tokenize:
        data_clean = [replaceSlang(word) for word in data]
        data_formal.append(data_clean)
    len_data_formal = len(data_formal)
    # print(data_formal)
    # len_data_formal
    
    nltk.download('stopwords')
    default_stop_words = nltk.corpus.stopwords.words('indonesian')
    stopwords = set(default_stop_words)

    def removeStopWords(line, stopwords):
        words = []
        for word in line:  
            word=str(word)
            word = word.strip()
            if word not in stopwords and word != "" and word != "&":
                words.append(word)

        return words
    data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]
    
    white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

    factory = StemmerFactory()
    ind_stemmer = factory.create_stemmer()
    def stemmer(line):
        temp = list()
        for word in line:
            if(word not in white_list):
                word = ind_stemmer.stem(word)
            if(len(word)>3):
                temp.append(word)
            return temp

    reviews = [stemmer (line) for line in data_notstopword]
    
    #Pembuatan Kamus kata
    t  = Tokenizer()
    fit_text = reviews
    t.fit_on_texts(fit_text)

    #Pembuatan Id masing-masing kata
    sequences = t.texts_to_sequences(reviews)

    #hapus duplikat kata yang muncul
    list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

    #mencari max length sequence
    def FindMaxLength(lst): 
        maxList = max((x) for x in lst) 
        maxLength = max(len(x) for x in lst ) 
        return maxList, maxLength 
        
    # Driver Code 
    max_seq, max_length_seq = FindMaxLength(list_set_sequence)
    jumlah_index = len(t.word_index) +1

    print('jumlah index : ',jumlah_index,'\n')
    # print('word_index : ',t.word_index,'\n')
    # print('index kalimat asli     : ', sequences,'\n')
    # print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
    # print('panjang max kalimat : ', max_length_seq,'kata','\n')
    # print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

    count_word = [len(i) for i in list_set_sequence]
    # print('list panjang kalimat : ', count_word)
    max_len_word = max(count_word)
    # print(max_len_word)
    
    padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')
    
    nama_model='gabungan'
    model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
    result=[]
    for test in padding:
        pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
        result.append(pred[0])
    
    df_ganjar['Tokoh'] = 'Ganjar Pranowo'
    df_ganjar['Sentiment'] = result
    df_ganjar.loc[df_ganjar['Sentiment'] == 2, 'Sentiment'] = -1
    df_ganjar.head()
    
    df_ganjar['Sentiment'].unique()
    
    def insert_to_list_rekomen_db(rp2):
        # Connect to the database
        connection = pymysql.connect(host='127.0.0.1',
                                    port=3306,
                                    user='root',
                                    # password='f#Ur8J3N',
                                    database='klasifikasi_sentimen')
        # create cursor
        cursor=connection.cursor()
        
        cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
        for i,row in rp2.iterrows():
            sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(sql, tuple(row))

            # the connection is not autocommitted by default, so we must commit to save our changes
            connection.commit()
            
    insert_to_list_rekomen_db(rp2=df_ganjar)
    
    start += 1000
    end += 1000

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  357 

1/1 [==============================] - 0s 60ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  364 

1/1 [==============================] - 0s 42ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

jumlah index :  454 

1/1 [==============================] - 0s 42ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  382 

1/1 [==============================] - 0s 23ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  373 

1/1 [==============================] - 0s 23ms/step


In [13]:
#menyimpan tweet. (tipe data series pandas)
data_content = df_ganjar['tweet']
# casefolding
data_casefolding = data_content.str.lower()
#data_casefolding.head()
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

#slang word
path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [14]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
# print(reviews)

In [17]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
# print('word_index : ',t.word_index,'\n')
# print('index kalimat asli     : ', sequences,'\n')
# print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
# print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
# print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
# print(max_len_word)

jumlah index :  2132 



In [18]:
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')

In [19]:
nama_model='gabungan'
model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
result=[]
for test in padding:
  pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
  result.append(pred[0])

1/1 [==============================] - 0s 50ms/step


In [20]:
df_ganjar['Tokoh'] = 'Ganjar Pranowo'
df_ganjar['Sentiment'] = result
df_ganjar.loc[df_ganjar['Sentiment'] == 2, 'Sentiment'] = -1
df_ganjar.head()

,date,username,tweet,location,Tokoh,Sentiment
0,2023-04-16 06:05:51,merry_gemoy,Ganjar pranowo berbagi kebahagiaan dengan anak...,ACEH,Ganjar Pranowo,-1
1,2023-04-16 06:05:36,merry_gemoy,Ganjar pranowo raih elektabilitas tinggi di Ja...,ACEH,Ganjar Pranowo,-1
2,2023-04-16 05:55:47,ongapican,@ch_chotimah2 @ganjarpranowo Lonteh pura2 ‘ali...,ACEH,Ganjar Pranowo,-1
3,2023-04-16 05:47:14,MOSLEMSOLIDARI1,@ch_chotimah2 @ganjarpranowo https://t.co/dTj...,ACEH,Ganjar Pranowo,1
4,2023-04-16 05:22:49,fanisalesti,Ganjar pranowo emang keren #GanjarBawaKemajua...,ACEH,Ganjar Pranowo,1


In [21]:
df_ganjar['Sentiment'].unique()

array([-1,  1,  0], dtype=int64)

In [22]:
def insert_to_list_rekomen_db(rp2):
    # Connect to the database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 # password='f#Ur8J3N',
                                 database='klasifikasi_sentimen')
    # create cursor
    cursor=connection.cursor()
    
    cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
    for i,row in rp2.iterrows():
        sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

In [23]:
insert_to_list_rekomen_db(rp2=df_ganjar)